# User inferer

A user-provided python file shown below is used to update the brails-created asset inventory

In [3]:
import os

cwd = os.getcwd()
filepath =  os.path.join(cwd,"content_value_inferer.py")

with open(filepath, 'r') as f:
    file_contents = f.read()
print(file_contents)

import numpy as np


def user_inferer(inventory_array):

	#
	# Defining my mapping following Table 6-10 in Hazus Inventory Technical Manual 6
	#
	# Baseline Hazus Contents Value as Percent of Structure Value

	contents_value_over_str_value = {
	    "RES1": 0.50,
	    "RES2": 0.50,
	    "RES3A": 0.50,
	    "RES3B": 0.50,
	    "RES3C": 0.50,
	    "RES3D": 0.50,
	    "RES3E": 0.50,
	    "RES3F": 0.50,
	    "RES3": 0.50,
	    "RES4": 0.50,
	    "RES5": 0.50,
	    "RES6": 0.50,
	    "COM1": 1.00,
	    "COM2": 1.00,
	    "COM3": 1.00,
	    "COM4": 1.00,
	    "COM5": 1.00,
	    "COM6": 1.50,
	    "COM7": 1.50,
	    "COM8": 1.00,
	    "COM9": 1.00,
	    "COM10": 0.50,
	    "IND1": 1.50,
	    "IND2": 1.50,
	    "IND3": 1.50,
	    "IND4": 1.50,
	    "IND5": 1.50,
	    "IND6": 1.00,
	    "AGR1": 1.00,
	    "REL1": 1.00,
	    "GOV1": 1.00,
	    "GOV2": 1.50,
	    "EDU1": 1.00,
	    "EDU2": 1.50
	}

	new_features = {}
	for idx,bldg in inventory_array.items():

		occ_type = bldg["properties"]["occup

The python file needs to contain a function named "user_inferer" which takes the current inventory (json format) as input and outputs the new features (json format). Below is example of the current inventory

## Generation of base inventory

In [4]:
import sys
import copy

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory
importer = Importer()

In [5]:
region_data = {"type": "locationName", "data": "Fort Myers Beach"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [6]:
#
# Get Footprints using OSM
#

print("Trying OSM_FootprintsScraper ...")

osm_class = importer.get_class("OSM_FootprintScraper")
osm_data = {"length": "ft"}
osm = osm_class(osm_data)
osm_inventory = osm.get_footprints(region_boundary_object)


Trying OSM_FootprintsScraper ...

Searching for Fort Myers Beach...
Found Fort Myers Beach, Lee County, Florida, 33931, United States

Found a total of 2768 building footprints in Fort Myers Beach


In [7]:
nsi_class = importer.get_class("NSI_Parser")
nsi = nsi_class()

In [8]:
my_inventory = nsi.get_filtered_data_given_inventory(osm_inventory, "ft")



Getting National Structure Inventory (NSI) building data for the entered location input...
Found a total of 2503 building points in NSI that match the footprint data.


In [9]:
my_inventory.get_asset_features(1)

(True,
 {'type': 'Building',
  'buildingheight': 'NA',
  'erabuilt': 'NA',
  'numstories': 'NA',
  'roofshape': 'NA',
  'fpAreas': 8238})

In [10]:
knn_imputer_class = importer.get_class("KnnImputer")
imputer=knn_imputer_class()
fort_myers_imputed = imputer.impute(my_inventory,n_possible_worlds=10, exclude_features=["lon","lat","fd_id"])

Features with no reference data cannot be imputed. Removing them from the imputation target: buildingheight
Missing percentages among 2768 assets
erabuilt: 9.57%
numstories: 9.57%
roofshape: 99.82%
fparea: 9.57%
repaircost: 9.57%
constype: 9.57%
occupancy: 9.57%
Primitive imputation done.
Running the main imputation. This may take a while.
Done imputation. It took 0.10 mins


In [11]:
fort_myers_imputed.get_asset_features(1)

(True,
 {'type': 'Building',
  'buildingheight': 'NA',
  'erabuilt': 1983.0,
  'numstories': 1.0,
  'roofshape': 'flat',
  'fpAreas': 8238,
  'fparea': [605.35504,
   1210.45,
   1210.45,
   372.847142,
   2987.0,
   605.35504,
   463.525678,
   1210.45,
   1210.45,
   1210.45],
  'repaircost': [212759.348,
   212759.348,
   190695.273,
   68733.4113,
   158685.666,
   212759.348,
   212759.348,
   212759.348,
   212759.348,
   190695.273],
  'constype': 'W1',
  'occupancy': ['RES1',
   'RES1',
   'RES1',
   'RES1',
   'RES1',
   'RES1',
   'RES1',
   'REL1',
   'RES1',
   'RES1']})

# Run user inferer to update the floor area

In [12]:
user_inferer_class = importer.get_class("UserInferer")
inferer=user_inferer_class()

fort_myers_inferred = inferer.infer(fort_myers_imputed,filepath)

All assets are updated


In [13]:
fort_myers_inferred.get_asset_features(55)

(True,
 {'type': 'Building',
  'buildingheight': 'NA',
  'erabuilt': [1973.0,
   1973.0,
   1973.0,
   1973.0,
   1973.0,
   1973.0,
   1973.0,
   1973.0,
   1973.0,
   1974.0],
  'numstories': 1.0,
  'roofshape': 'flat',
  'fpAreas': 564,
  'fparea': 1210.45,
  'repaircost': 212759.348,
  'constype': ['W1', 'C1', 'W1', 'C1', 'W1', 'W1', 'W1', 'C1', 'W1', 'W1'],
  'occupancy': ['COM4',
   'RES3A',
   'COM4',
   'COM4',
   'RES1',
   'COM4',
   'RES1',
   'RES1',
   'COM4',
   'RES3A'],
  'contentsValue': [212759.348,
   106379.674,
   212759.348,
   212759.348,
   106379.674,
   212759.348,
   106379.674,
   106379.674,
   212759.348,
   106379.674]})

### Note you can also avoid the overwritting of already existing values

In [14]:
fort_myers_inferred = inferer.infer(fort_myers_imputed, filepath,overwrite=False)

All assets are updated


In [15]:
fort_myers_inferred.get_asset_features(1)

(True,
 {'type': 'Building',
  'buildingheight': 'NA',
  'erabuilt': 1983.0,
  'numstories': 1.0,
  'roofshape': 'flat',
  'fpAreas': 8238,
  'fparea': [605.35504,
   1210.45,
   1210.45,
   372.847142,
   2987.0,
   605.35504,
   463.525678,
   1210.45,
   1210.45,
   1210.45],
  'repaircost': [212759.348,
   212759.348,
   190695.273,
   68733.4113,
   158685.666,
   212759.348,
   212759.348,
   212759.348,
   212759.348,
   190695.273],
  'constype': 'W1',
  'occupancy': ['RES1',
   'RES1',
   'RES1',
   'RES1',
   'RES1',
   'RES1',
   'RES1',
   'REL1',
   'RES1',
   'RES1'],
  'contentsValue': [106379.674,
   106379.674,
   95347.6365,
   34366.70565,
   79342.833,
   106379.674,
   106379.674,
   212759.348,
   106379.674,
   95347.6365]})